In [ ]:
%pip install aiohttp

## XML Parser
This notebook consumes **Delta CDF** from `submissions` where `status='content_extraction_completed'`, calls a **parse_xml_content api**

## Imports

## ConfigManager

In [ ]:
"""
Configuration management utilities for PDF processing framework.

This module provides utilities for loading, validating, and managing
configuration files used throughout the PDF processing pipeline.
"""

import yaml
import os
import time
from typing import Dict, Any, List, Optional, Union
from pathlib import Path
from pyspark.sql import Window
from pyspark.sql import functions as F


class ConfigManager:
    """
    Configuration manager for handling YAML and JSON configuration files.
    
    Provides methods for:
    - Loading configuration from files
    - Validating configuration structure
    - Merging configurations
    - Environment variable substitution
    """
    
    def __init__(self, config_path: Optional[Union[str, Path]] = None):
        """
        Initialize configuration manager.
        
        Args:
            config_path: Path to configuration file
            config_dict: Configuration dictionary (alternative to file)
            
        Raises:
            ValueError: If neither config_path nor config_dict is provided
        """
        if config_path:
            self.config = self.load_config_file(config_path)
            self.config_path = str(config_path)
        else:
            raise ValueError("Either config_path or config_dict must be provided")
        
        # Substitute environment variables
        # self._substitute_environment_variables()

    @staticmethod
    def get_file_extension(file_path: Union[str, Path]) -> str:
        """
        Get file extension (without dot).
        
        Args:
            file_path: Path to file
            
        Returns:
            File extension in lowercase
        """
        return Path(file_path).suffix.lower().lstrip('.')
    
    @staticmethod
    def load_config_file(file_path: Union[str, Path]) -> Dict[str, Any]:
        """
        Load configuration from YAML or JSON file.
        
        Args:
            file_path: Path to configuration file
            
        Returns:
            Configuration dictionary
            
        Raises:
            FileNotFoundError: If file doesn't exist
            ValueError: If file format is not supported
        """
        extension = ConfigManager.get_file_extension(file_path)
        
        if extension in ['yaml', 'yml']:
            with open(file_path, 'r', encoding='utf-8') as f:
                return yaml.safe_load(f) or {}
        else:
            raise ValueError(f"Unsupported configuration file format: {extension}")
    
    def get(self, key: str, default: Any = None) -> Any:
        """
        Get configuration value with optional default.
        
        Args:
            key: Configuration key (supports dot notation for nested keys)
            default: Default value if key is not found
            
        Returns:
            Configuration value or default
        """
        return self._get_nested_value(self.config, key, default)
        
    @staticmethod
    def _get_nested_value(config: Dict[str, Any], key: str, default: Any = None) -> Any:
        """
        Get nested configuration value using dot notation.
        
        Args:
            config: Configuration dictionary
            key: Key with dot notation (e.g., 'task.params.value')
            default: Default value if key is not found
            
        Returns:
            Configuration value or default
        """
        keys = key.split('.')
        value = config
        
        for k in keys:
            if isinstance(value, dict) and k in value:
                value = value[k]
            else:
                return default
        
        return value
        
    def to_dict(self) -> Dict[str, Any]:
        """
        Get configuration as dictionary.
        
        Returns:
            Configuration dictionary
        """
        return self.config.copy()
    

    def get_pipeline_steps(config: dict, pipeline_name: str, document_format: str, step_name: str) -> dict:
        """
        Return the steps dictionary for a given pipeline and document format.
        """
        step_list = config.get(pipeline_name, {}).get(document_format, [])
        step_config = ConfigManager.get_step_config(step_list, step_name)
        return step_config


    @staticmethod
    def get_step_config(config: list, step_name: str) -> dict:
        """
        Fetch all variables for a given step from the workflow config.

        Args:
            config (list): List of step dictionaries.
            step_name (str): Step name to search for.

        Returns:
            dict: Config dictionary for the step, or {} if not found.
        """
        for step in config:
            if step.get("step") == step_name:
                return step
        return {}

   


## Get the task variables required for the workflow

In [ ]:
# ---------------- CONFIGURATION ----------------

import json
from pyspark.sql.types import StructType, StructField, StringType, LongType, TimestampType, IntegerType, BooleanType

environment = dbutils.jobs.taskValues.get(
    taskKey="unified_workflow_setup", key="environment")
job_id = dbutils.jobs.taskValues.get(
    taskKey="unified_workflow_setup", key="databricks_job_id")
job_run_id = dbutils.jobs.taskValues.get(
    taskKey="unified_workflow_setup", key="databricks_job_run_id")
# TODO: Pass from the config
pipeline_id = "acordis"

# Load configuration sections that were processed by ConfigManager in unified_setup
globals_config = json.loads(dbutils.jobs.taskValues.get(
    taskKey="unified_workflow_setup", key="globals_config"))
services_config = json.loads(dbutils.jobs.taskValues.get(
    taskKey="unified_workflow_setup", key="services_config"))
pipelines_config = json.loads(dbutils.jobs.taskValues.get(
    taskKey="unified_workflow_setup", key="pipelines_config"))

# Table configuration using ConfigManager-processed values
catalog = globals_config["catalog"]
bronze_schema_name = globals_config["bronze_schema_name"]
silver_schema_name = globals_config["silver_schema_name"]

xmlparser_cfg = ConfigManager.get_pipeline_steps(
    pipelines_config, pipeline_id, "pdf", "XMLParser")
output_submission_status = xmlparser_cfg["submission_status"]
submissions_table = submission_table = f"{catalog}.{bronze_schema_name}.{xmlparser_cfg["submissions_table"]}"
document_payloads_table = f"{catalog}.{silver_schema_name}.{xmlparser_cfg["document_payloads_table"]}"
output_table = f"{catalog}.{silver_schema_name}.{xmlparser_cfg["output_table"]}"

synthetic_mode = xmlparser_cfg["synthetic_mode"]
micro_batch_limit = xmlparser_cfg["micro_batch_limit"]
target_partitions = xmlparser_cfg["target_partitions"]

# Streaming configuration for scalability
checkpoint_base_location = globals_config["checkpoint_base_location"]
checkpoint_location = f"{checkpoint_base_location}/xml_parsing_checkpoints"

# Scalability parameters
# num_partitions = xmlparser_cfg["num_partitions"]
max_files_per_trigger = globals_config["default_batch_size"]

print(" Configuration loaded:")
print(f"    submissions_table: {submissions_table}")
print(f"    output_table: {output_table}")
print(f"    Checkpoint: {checkpoint_location}")
print(f"    Max files per trigger: {max_files_per_trigger}")
print(f"    Synthetic mode: {synthetic_mode}")
print(f"    Job ID: {job_id}, Run ID: {job_run_id}")

result_schema = StructType([
    StructField("submission_id", StringType(), False),
    StructField("node_id", IntegerType(), True),
    StructField("node_page", IntegerType(), True),
    StructField("node_bounding_box", StringType(), True),
    StructField("node_header", StringType(), True),
    StructField("node_parent", IntegerType(), True),
    StructField("node_content", StringType(), True),
    StructField("node_content_xml", StringType(), True),
    StructField("node_level", IntegerType(), True),
    StructField("node_class", StringType(), True),
    StructField("image_base64encoded_string", StringType(), True),
    StructField("error_traceback", StringType(), True),
    StructField("parsing_status", StringType(), True),
    StructField("processing_time_ms", LongType(), True),
    StructField("parsing_timestamp", TimestampType(), True),
    StructField("is_header", BooleanType(), True)                 
])

In [ ]:
# Initialize framework step executor for XML parsing
# Create step executor instance with synthetic mode flag
# step_executor = StepExecutor(use_synthetic=synthetic_mode) 

# print("✅ Using XML parsing from framework StepExecutor")
# print(f"   - Synthetic mode: {synthetic_mode}")
# print("   - Framework provides parse_xml_content method")
# print("   - Scalable and reusable across workflows")



## Read Source Data
Read data from submissions table for submission_status=content_extraction_completed

In [ ]:

# CDF-based streaming source for automatic change detection
source_stream = (
    spark.readStream
         .format("delta")
         .option("readChangeFeed", "true")  # Enable CDF
         .table(submissions_table)
         .where(F.col("_change_type").isin("insert", "update_postimage"))
         .where(F.col("submission_status") == "content_extraction_completed")
         .selectExpr("* EXCEPT(_change_type, _commit_version, _commit_timestamp)")
)

submissions_stream = source_stream.filter(F.col("pipeline_id") == "acodis")

print(" CDF streaming source configured")
print("   - Automatically detects status changes")
print("   - Event-driven processing (no polling)")
print("   - Exactly-once processing guarantees")


### process_partition

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.types import (
    StructType, StructField, StringType, LongType, TimestampType
)
import pandas as pd
from datetime import datetime
import json

import re
import xml.etree.ElementTree as ET
from copy import deepcopy
import ast
import traceback


def extract_text(element: ET.Element) -> str:
    """
    Recursively extract text from an element's subtree, including nested elements.

    Args:
        element (ET.Element): Element from which to extract text.

    Returns:
        str: Concatenated text collected from the element and its subtree.

    """
    text_parts = []
    if element.text:
        text_parts.append(element.text.strip())
    for child in element:
        text_parts.append(extract_text(child))
        if child.tail:
            text_parts.append(child.tail.strip())
    return " ".join(text_parts)


def process_list_elements(ul_ol):
    """
    Process and format XML list elements.

    Args:
        ul_ol (ET.Element): The XML list element to be processed.

    Returns:
        str: A string representation of the processed list element.

    """
    list_items = []
    for li in ul_ol.findall("li"):
        li_content = []
        data_enum = li.get("data-enum", "").strip()
        for child in li:
            li_content.append(extract_text(child))  # noqa: PERF401 - append to the list
        nested_lists = li.findall("ul") + li.findall("ol")
        for nested_list in nested_lists:
            li_content.append(process_list_elements(nested_list))  # noqa: PERF401  -append to the list
        formatted_content = f"{data_enum} {' '.join(li_content).strip()}"
        list_items.append(formatted_content)

    return "\n".join(list_items)


# TODO: Lower the cognitive complexity of this function in future through https://jira.jnj.com/browse/JEDU-2367.
def process_table(table) -> str:
    """
    Process and format an XML table element into a Markdown table.

    Args:
        table: An XML table element.

    Returns:
        str: A formatted string representation of the table in Markdown.

    """
    table_content = []
    rowspan_tracker = {}
    max_cols = 0
    for r_idx, tr in enumerate(table.findall("tr")):
        current_row = []
        col_idx = 0
        # Fill placeholders from rowspan_tracker if present
        while col_idx < len(current_row):
            if (r_idx, col_idx) in rowspan_tracker:
                current_row.append(rowspan_tracker[(r_idx, col_idx)])
                rowspan_tracker[(r_idx, col_idx)] -= 1
                if rowspan_tracker[(r_idx, col_idx)] == 0:
                    del rowspan_tracker[(r_idx, col_idx)]
                col_idx += 1
        for td in tr.findall("td"):
            # Fill placeholders from rowspan_tracker in the current row
            while (r_idx, col_idx) in rowspan_tracker:
                current_row.append("")
                rowspan_tracker[(r_idx, col_idx)] -= 1
                if rowspan_tracker[(r_idx, col_idx)] == 0:
                    del rowspan_tracker[(r_idx, col_idx)]
                col_idx += 1
            # Get the cell text
            node_content, _ = process_children(td)
            cell_text = " ".join(node_content).strip() if td is not None else ""
            # Place the text in the current row
            current_row.append(cell_text)
            rowspan = int(td.get("rowspan", 1))
            # If there's a rowspan, track it for future rows
            if rowspan > 1:
                for i in range(1, rowspan):
                    rowspan_tracker[(r_idx + i, col_idx)] = rowspan - i
            col_idx += 1
        max_cols = max(max_cols, len(current_row))
        table_content.append(current_row)
        # Normalize all rows to have the same column count
    for row in table_content[1::]:
        while len(row) < max_cols:
            row.append(" ")

    # Generate Markdown table
    header = table_content[0]
    separator = ["---"] * max_cols
    body = table_content[1:]

    # Combine header, separator, and body into a Markdown table
    markdown_table = (
        "| "
        + " | ".join(header)
        + " |\n"
        + "| "
        + " | ".join(separator)
        + " |\n"
        + "\n".join(["| " + " | ".join(row) + " |" for row in body])
    )
    return markdown_table


# TODO: Lower the cognitive complexity of this function in future through https://jira.jnj.com/browse/JEDU-2367.
def process_children(element):
    """
    Process and format XML element's children.

    Args:
        element (ET.Element): The XML element whose children are to be processed.

    Returns:
        list: A list containing strings of the processed children elements.

    """
    children_content = []
    if element.tag != "figure":
        node_class = "text"
        for child in element:
            if child.tag == "p":
                text = "".join(child.itertext()).strip()
                text = re.sub(r"\s+", " ", text)
                if children_content:
                    children_content.append(" " + text)
                else:
                    children_content.append(text)
            elif child.tag in ["ul", "ol"]:
                children_content.append(process_list_elements(child))
            elif child.tag == "table":
                children_content.append(process_table(child))
    else:
        for child in element:
            if child.tag == "img":
                children_content.append(child.get("src", ""))
                node_class = "image"
    return children_content, node_class


def remove_metadata(xml_string: str) -> str:
    """
    Remove metadata from xml tags.

    Args:
        xml_string (str): XML content for which the metadata must be removed.

    Returns:
        str: Cleaned XML string.

    """
    root = ET.fromstring(xml_string)  # noqa: S314  -extract the xml string
    for elem in root.iter():
        elem.attrib.clear()
    cleaned_xml = ET.tostring(root, encoding="unicode")
    return cleaned_xml


def serialize_section_without_subsections(section):
    """
    Serialize an XML section without including nested sections.

    Args:
        section: Section to be serialized.

    Returns:
        str: Serialized XML section

    """
    section_copy = ET.Element(section.tag, attrib=section.attrib)

    for child in section:
        if child.tag == "section":
            continue
        section_copy.append(deepcopy(child))
    return ET.tostring(section_copy, encoding="unicode")


def get_parent_child_nodes(dataframe):
    """
    Generate the parent_child hierarchy.

    Arguments:
        dataframe (DataFrame): DataFrame with a new column called 'node_parent_id'.

    Returns:
        DataFrame: The modified DataFrame including the 'node_parent_id' column.

    """
    # Create a dictionary to map node_id to id

    
    id_map = dataframe.set_index("node_id")[
        "id"
    ].to_dict()

    dataframe["node_parent_id"] = dataframe.apply(
        lambda row: -1
        if row["node_parent"] == "-1"
        else id_map.get(row["node_parent"], None),
        axis=1,
    )
    
    dataframe["node_parent_id"] = (
        pd.to_numeric(
            dataframe["node_parent_id"], errors="coerce"
        )
        .fillna(-1)
        .astype(int)
    )
    return dataframe


def format_xml_content_for_view_document(
    xml_file_path: str,
    response_text: str,
    document_title: str,
    document_id: int,
) -> list:
    """
    Transform the XML file content to a structured format, compatible with the frontend's view document functionality.

    Args:
        db (Session): The Database Session.
        xml_file_path (str): The path to the XML file.
        response_text (str): The content of the XML file.
        document_title (str): The document title.
        document_id (int): Document Id.

    Returns:
        list: A list of dictionaries containing node_source, node_id, node_page, node_bounding_box, bounding_box,
        node_header, node_parent and node_content.

    """
    root = ET.fromstring(response_text)  # noqa: S314   -Extract the xml string
    result = []
    for section in root.findall("section"):
        result = process_xml_section(
            section,
            xml_file_path,
            result=result,
            document_title=document_title,
            document_id=document_id,
        )
    return result


def process_xml_section(  # noqa: PLR0913   -required arguments as per implementation
    section,
    xml_file_path: str,
    document_title: str,
    document_id: int,
    parent_id: int = -1,
    node_id_counter: list | None = None,
    node_headers: dict | None = None,
    result: list | None = None,
) -> list:
    """
    Recursively process XML sections and extract necessary data.

    Args:
        section: An XML section element.
        xml_file_path (str): The URL path to the XML file.
        document_title (str): The document's title.
        db (Session): The Database Session.
        document_id (int): The document id.
        parent_id (int, optional): ID of the parent node. Defaults to -1.
        node_id_counter (list, optional): List to hold the current node ID. Defaults to [-1].
        node_headers (dict, optional): Dictionary to keep track of headers for each node. Defaults to {}.
        result (list, optional): The list to which the results will be appended.

    Returns:
        list: Updated list with the results.

    """
    if node_id_counter is None:
        node_id_counter = [-1]
    if node_headers is None:
        node_headers = {}
    if result is None:
        result = []
    node_id_counter[0] += 1
    current_id = node_id_counter[0]
    header_element = section.find("header")
    header = (
        "".join(header_element.itertext()).strip()
        if header_element is not None
        else ""
    )
    level = int(section.get("level", 0))
    page = int(section.get("page", 0))
    bounding_box = [
        float(section.get(attr, 0.0))
        for attr in ["left", "top", "width", "height"]
    ]
    node_header = {level: header}
    image_base64encoded_string: str | None = None
    if parent_id != -1:
        node_header.update(node_headers.get(parent_id, {}))
    node_header = dict(sorted(node_header.items()))
    node_headers[current_id] = node_header
    section_node_content, node_class = process_children(section)
    node_content = "\n".join(section_node_content)
    if not node_content:
        node_content = header
        node_content_xml = (
            ET.tostring(header_element, encoding="unicode")
            if header_element is not None
            else ""
        )
        cleaned_node_content_xml = remove_metadata(node_content_xml)
    elif node_class != "image":
        node_content = f"{header} \n\n{node_content}"
        node_content_xml = serialize_section_without_subsections(section)
        cleaned_node_content_xml = remove_metadata(node_content_xml)
    elif node_class == "image":
        #print("no image processing yet......")
        return result


    result.append(
        {
            "node_source": document_title,
            "node_id": current_id,
            "node_page": page,
            "node_bounding_box": str(bounding_box),
            "node_header": str(list(node_header.values())),
            "node_parent": parent_id,
            "node_content": node_content,
            "node_content_xml": cleaned_node_content_xml,
            "node_level": level,
            "node_class": node_class,
            "image_base64encoded_string": image_base64encoded_string,
        }
    )

    for subsection in section.findall("./*"):
        if subsection.tag in ("section", "figure"):
            process_xml_section(
                subsection,
                xml_file_path,
                document_title=document_title,
                parent_id=current_id,
                node_id_counter=node_id_counter,
                node_headers=node_headers,
                result=result,
                document_id=document_id,
            )

    return result

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.types import (
    StructType, StructField, StringType, LongType, TimestampType
)
import pandas as pd
from datetime import datetime
import json


def process_partition(rows_iter):
    """
    mapInPandas iterator. For each pandas DataFrame (a partition batch),
    call your parse_xml_content() row by row and yield a pandas DataFrame
    matching output_schema.
    """
    for row_df in rows_iter:
        out_rows = []
        cols = result_schema.names
        now = datetime.now()
        new_submission_status = output_submission_status
        if row_df.empty:
            yield pd.DataFrame(columns=cols)
            continue

        for _, row in row_df.iterrows():
            start_time = time.time()
            submission_id = row["submission_id"]
            download_path = row["download_path"]
            file_name = row['file_path'].split('/')[-1]

            try:
                with open(download_path, 'r', encoding='utf-8') as file:
                    xml_content = file.read()
                
                # Parse XML and extract structured data
                root = ET.fromstring(xml_content)
                
                result = []
                for section in root.findall("section"):
                    result = process_xml_section(
                        section,
                        download_path,
                        result=result,
                        # document title becomes the id
                        document_title=submission_id,
                        document_id=file_name
                    )

                processing_time_ms = int((time.time() - start_time) * 1000)
                table = pd.DataFrame(result)

                table["is_header"] = table.apply(
                    lambda row: row["node_content"].strip()
                    == ast.literal_eval(row["node_header"])[-1].strip()
                    if isinstance(row["node_header"], str)
                    and row["node_header"].startswith("[")
                    and row["node_header"].endswith("]")
                    else False,
                    axis=1,
                )

                table["node_content_xml"] = table.apply(
                    lambda row: (
                        row["node_content_xml"]
                        .replace("<section>", "")
                        .replace("</section>", "")
                    )
                    if row["node_class"] != "image"
                    else row["node_content_xml"],
                    axis=1,
                )

                table.rename(columns={'node_source': 'submission_id'}, inplace=True)
                if "error_traceback" not in table.columns.tolist():
                    table['error_traceback'] = None
                if "parsing_status" not in table.columns.tolist():
                    table['parsing_status'] = 'success'
                if "processing_time_ms" not in table.columns.tolist():
                    table['processing_time_ms'] = processing_time_ms
                if "parsing_timestamp" not in table.columns.tolist():
                    table['parsing_timestamp'] = now
            except:
                table = pd.DataFrame({
                    "submission_id": [submission_id],
                    "node_id": [None],
                    "node_page": [None],
                    "node_bounding_box": [None],
                    "node_header": [None],
                    "node_parent": [None],
                    "node_content": [None],
                    "node_content_xml": [None],
                    "node_level": [None],
                    "node_class": [None],
                    "image_base64encoded_string": [None],
                    "error_traceback": [traceback.format_exc()],
                    "parsing_status": ["error"],
                    "processing_time_ms": [None],
                    "parsing_timestamp": [now],
                    "is_header": [None]
                })

        yield table


## foreach_batch

In [ ]:
def foreach_batch(batch_df, batch_id):
    """
    Serverless-safe: uses DataFrame API + mapInPandas (no RDDs), and set-based SQL to update statuses.
    Expects globals: source_tbl, document_payloads_table, target_tbl, num_partitions, job_id, job_run_id
    """
    if batch_df.isEmpty():
        print(f"→ Batch {batch_id}: empty")
        return

    spark = batch_df.sparkSession

    # Deduplicate by latest per submission_id inside the microbatch 
    w = Window.partitionBy("submission_id").orderBy(
        F.col("last_updated").desc_nulls_last())
    latest = (batch_df
              .withColumn("rn", F.row_number().over(w))
              .where("rn = 1")
              .drop("rn"))


    # Optional claim step to prevent parallel runs from double-processing:
    # Create a temp view and atomically mark a slice as 'claimed' by this job.
    latest.createOrReplaceTempView("micro_latest")

    limit_clause = f"LIMIT {int(micro_batch_limit)}" if micro_batch_limit else "" 
    spark.sql(f"""
      MERGE INTO {submission_table} t
      USING (
        SELECT submission_id
        FROM micro_latest
        WHERE submission_status IN ('content_extraction_completed')
        {limit_clause}
      ) c
      ON t.submission_id = c.submission_id
      WHEN MATCHED THEN UPDATE SET
        t.submission_status = 'xml_parsing_in_progress',
        t.last_updated      = current_timestamp()
    """)

    # Load only the rows claimed by this run id (ensures single consumer per row)
    claimed = (spark.read.table(submission_table)
               .where(F.col("submission_status") == "xml_parsing_in_progress"))

    claimed_enriched_df = (
        claimed.alias("b")
        .join(F.broadcast(spark.table(document_payloads_table).alias("p")),
              on=F.col("b.submission_id") == F.col("p.submission_id"),
              how="inner")
        .select(
            F.col("p.download_path"),
            F.col("b.submission_id"),
            F.col("b.file_path")
        )   
    )

    if claimed_enriched_df.count() == 0:
        print(f"→ Batch {batch_id}: No rows claimed")
        return
    
    print(f"processing {claimed_enriched_df.count()} records with id : {batch_id}")

    # Repartition for wide fan-out, but keep skew keys (submission_id) as the partition column
    claimed_enriched_df = claimed_enriched_df.repartition(target_partitions, F.col("submission_id"))

    # MapPartitions on executors to call REST & write files
    result_df = claimed_enriched_df.mapInPandas(process_partition, schema=result_schema)
    result_df = result_df.withColumn("databricks_parsing_job_id", F.lit(job_id))
    result_df = result_df.withColumn("databricks_parsing_job_run_id", F.lit(job_run_id))
    result_df = result_df.withColumn("parsing_batch_id", F.lit(batch_id).cast("string"))
    result_df.write.format("delta").mode("append").saveAsTable(output_table)

    # update the status of the submission
    result_df.createOrReplaceTempView("xml_results_batch")
    # MERGE results back, clear claim, set result metadata
    # success
    spark.sql(f"""
        UPDATE {submissions_table} AS s
        SET    s.submission_status = '{output_submission_status}',
                s.last_updated = current_timestamp()
        WHERE EXISTS (
        SELECT 1
        FROM xml_results_batch r
        WHERE r.submission_id = s.submission_id
            AND r.parsing_status = 'success'
        )
    """)

    # failed
    spark.sql(f"""
        UPDATE {submissions_table} AS s
        SET    s.submission_status = 'xml_parsing_failed',
                s.last_updated = current_timestamp()
        WHERE EXISTS (
        SELECT 1
        FROM xml_results_batch r
        WHERE r.submission_id = s.submission_id
            AND r.parsing_status = 'error'
        )
    """)



## Write Data 
1. Write data to silver table
2. On success, update submission_status=xml_parsing_completed

In [ ]:
print("🚀 Starting scalable XML parsing streaming job...")

streaming_query = (submissions_stream.writeStream
                   .foreachBatch(foreach_batch)
                   .option("checkpointLocation", checkpoint_location)
                   .option("maxFilesPerTrigger", max_files_per_trigger)
                   .trigger(availableNow=True)  # Process all available data then stop
                   .outputMode("append")
                   .start())

print("XML Parsing Job Completed started")

try:
    streaming_query.awaitTermination()
    print("XML Parsing Job Completed completed successfully")    
    
except Exception as e:
    print(f"Streaming job error: {str(e)}")
    streaming_query.stop()
    raise
